### ДЗ к Урок 1. Введение в задачу предсказания оттока. Формализация задачи и сбор сырых данных

##### 1.Согласно принципу SMART, попробуйте расписать кейс кредитного скоринга, разбиравшийся на одном из предыдущих курсов.

__S__	Specific (Конкретность) - решаем задачу бинарной классификации, разрабатываем модель логистической регрессии для решения о выдаче кредита;

__M__	Measurable (Измеримость) - целевая переменная - решение о выдаче кредита, 1 - выдать кредит, 0 - отказать в кредите;

__A__	Attainable (Достижимость) - построение модели предполагает сбор данных, их подготовка, настройка модели, предсказание для теста и оценка качества;

__R__	Relevant (Актуальность/Уместность) - нужные данные есть, задача соответствует цели, ресурсы выделены под эту задачу;

__T__	Time-bound (Ограниченность во времени) - дается, условно, 1 месяц, чтобы собрать базовое решение и оценить перспективность дальнейшей разработки.


##### 2.Предложить перечень индикаторов оттока для клиента каршеринга. Какие данные мы бы взяли в датасет, предполагая, что в нашем распоряжении есть какие угодно сведения?

* дата регистрации
* дата первой аренды авто
* дата последней аренды авто
* дата рождения
* уровень использования (продолжительность поездки)
* общая сумма внесенных денег (LTV)
* пол
* страна
* привязан ли номер телефона
* платежная активность
* активность (количество поездок)
* жалобы на предыдущих пользователей авто
* технические проблемы (поломки авто)
* время аренды разных авто
* успешность поездок (без ДТП)
* оплата дополнительных услуг
* ...